In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import time

In [2]:
def get_review_url(product_url):
    # Extract ASIN from the product URL
    pattern = r'/dp/([A-Za-z0-9]+)'
    match = re.search(pattern, product_url)
    if match:
        asin = match.group(1)
        # Generate the review URL
        review_url = f'https://www.amazon.in/product-reviews/{asin}/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
        return review_url
    else:
        return "Invalid URL"

In [3]:
def get_page_count(review_url):
    page = requests.get(review_url)
    soup = bs(page.content, 'html.parser')
    page_count_string = (soup.find_all('div', {
        'data-hook': 'cr-filter-info-review-rating-count'}))[0].text.strip().split(' ')[3]
    page_count = int((int(page_count_string.replace(',', '')))//10 + 1)
    return page_count

In [4]:
def extract_reviews(reviews_list, review_url, page_count):
        
    review_url = review_url + f'&pageNumber={page_count}'
    soup = bs(requests.get(review_url).content, 'html.parser')
    reviews = soup.find_all('div', {'data-hook': 'review'})
    for review in reviews:
        review_dict = {
            'Page': page_count,
            'Review Title': (review.find_all('a', {'data-hook': 'review-title'}))[0].text.strip(),
            'Review Description': (review.find_all('span', {'data-hook': 'review-body'}))[0].text.strip(),
            'Rating': (review.find_all('i', {'data-hook': 'review-star-rating'})[0].text.strip())
        }

        reviews_list.append(review_dict)

In [5]:
def export_to_csv(reviews_list):
    df = pd.DataFrame(reviews_list)
    df.to_csv('reviews.csv', index=False)

In [12]:
product_url = 'https://www.amazon.in/Samsung-Galaxy-Cloud-128GB-Storage/dp/B08VB2MRF8/?_encoding=UTF8&pd_rd_w=DJDB2&content-id=amzn1.sym.aff93425-4e25-4d86-babd-0fa9faf7ca5d&pf_rd_p=aff93425-4e25-4d86-babd-0fa9faf7ca5d&pf_rd_r=HCH45Z56F02AF8F07CWQ&pd_rd_wg=zW6Li&pd_rd_r=5844dd9b-fd08-4f7d-96f7-1b0dce96040f&ref_=pd_gw_ci_mcx_mr_hp_atf_m&th=1'

review_url = get_review_url(product_url)
page_count = get_page_count(review_url)
print(page_count)
reviews_list = []

for i in range(1, page_count+1):
    extract_reviews(reviews_list, review_url, i)
print(reviews_list)
export_to_csv(reviews_list)

644
[]
